In [9]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from time import sleep
import random

In [14]:
for i in range(50):
    print(max(random.gauss(3,.7), 1.5))

2.471523243276347
3.0462965278895116
2.481521202145327
2.485539703064329
3.5365046164345095
1.5
3.1842698294738407
4.218589957590469
4.2024872816003445
1.5
2.9091046670861958
3.632776556422674
3.0037511022608756
2.6215740926961137
2.604362824606408
2.2322924713083334
2.375570258222158
2.4571039087765567
3.0669534542101413
4.544065669184519
4.2130936325110095
3.766216755160127
2.8984888150494053
4.023977324935917
3.316092485037279
1.9354372178076842
2.96491953035245
3.508578182278579
2.386284025082921
2.138985479294654
1.5
2.427739207573673
2.6885068179059135
4.132909496156168
1.9618392035467098
2.883700647927578
2.492805518547069
1.7080132686118779
3.218273204771964
3.2668336346021207
1.913081947882875
3.2133775552135693
3.547221002942473
4.005889258708855
2.7207948803413986
3.370473176683702
3.0675000662681637
3.358272089869131
3.1119055941946208
3.0202005123880733


3.082640781533935

### Exemple avec read_html

In [2]:
atp = 'https://fr.wikipedia.org/wiki/ATP_World_Tour'
# Exemple : extraction du 9ème tableau d'une pge Wikipedia
pd.read_html(atp)[8]

,Record,Joueur,Nombre,Précision
0,Titres,Titres,Titres,Titres
1,Total[5],Jimmy Connors,109,Jimmy Connors a gagné d'autres titres non reco...
2,En Grand Chelem[6],Rafael Nadal,21,entre 2005 et 2022
3,Aux Masters,Roger Federer,6,"2003, 2004, 2006, 2007, 2010 et 2011"
4,En Masters 1000[7] (créé en 1990),Novak Djokovic,37,entre 2007 et 2021
...,...,...,...,...
93,Points ATP en fin d'année,Novak Djokovic,16 585 points,2015
94,Points ATP (de 2000 à 2008),Roger Federer,8 370 points,20 novembre 2006
95,Points ATP en fin d'année (de 2000 à 2008),Roger Federer,8 370 points,2006
96,Points ATP (de 1994 à 1999),Pete Sampras,5 792 points,8 novembre 1997


#### Mini fonction servant à extraire des chiffres d'une chaine de caractères

In [3]:
def extract_digits(s):
    return int(''.join([x for x in s if x.isdigit()]))

#### Urls de la page à scrapper

In [4]:
imdb_page = 'https://www.imdb.com/search/title/?year=2019&title_type=feature&sort=boxoffice_gross_us,desc'
root_url = 'https://www.imdb.com'

#### Fonction destinée à récolter les infos d'une page

In [5]:
def getPageInfo(soup):
#     La liste vide qui se constituera petit à petit avec les éléments de la page
    movie_list = []
#     On boucle pour chaque bloc
    for el in soup.select('.lister-list .lister-item'):
        index_ =  el.select('.lister-item-index')[0].getText().replace('.', '')
        link =  el.select('.lister-item-header a')[0].get('href')
        title =  el.select('.lister-item-header a')[0].getText()
#         Conditionalité : pour ne ps faire planter le script si l'élément n'est pas trouvé (if ... else)
        year = extract_digits(el.select('.lister-item-year')[0].getText()) if el.select('.lister-item-year')[0].getText() else ''
        certificate = el.select('.certificate')[0].getText() if el.select('.certificate') else ''
        genre = el.select('.genre')[0].getText().strip() if el.select('.genre') else ''
        runtime = extract_digits(el.select('.runtime')[0].getText()) if el.select('.runtime') else ''
        ratings = el.select('.ratings-imdb-rating')[0].getText().strip() if el .select('.ratings-imdb-rating') else ''
        plot = el.select('.ratings-bar')[0].findNext('p').getText().strip() if el.select('.ratings-bar') else ''
        if el.select('.sort-num_votes-visible'):
#             Find + findNext : trouver le premier élément "span" après le texte "Votes:"
            votes = el.select('.sort-num_votes-visible')[0].find(text="Votes:").findNext('span').getText() if el.select('.sort-num_votes-visible')[0].find(text="Votes:") else ''
            gross = el.select('.sort-num_votes-visible')[0].find(text="Gross:").findNext('span').getText() if el.select('.sort-num_votes-visible')[0].find(text="Gross:") else ''
        else:
            votes = ''
            gross = ''
#         Le dictionaire qui vient s'ajouter à la liste
        dict_movie = {'index_': index_, 'link': link, 'title': title,
                     'year': year, 'certificate': certificate, 'genre': genre,
                     'runtime': runtime, 'ratings': ratings, 'plot': plot,
                     'votes': votes,'gross':gross}
        movie_list.append(dict_movie)
    return movie_list

#### Fonction récursive (qui s'appelle elle-même) qui appellera à chaque fois la page d'après si elle trouve un bouton "Next »"

In [6]:
def getNextlink(url, list_movie):
#     Importation du contenu de la page
    r = requests.get(url)
#     Parsing du contenu de la page
    soup = BeautifulSoup(r.text, 'lxml')
    list_movie_element = getPageInfo(soup)
    list_movie = list_movie + list_movie_element
    full_list['list_movie'] = list_movie
#     Si le bouton next est présent dans la page, la fonction s'appelle elle-même
    if soup.select('.desc')[0].find(text="Next »"):
        next_link = soup.select('.desc')[0].find(text="Next »").parent.get('href')
        print(next_link)
#         On attend une demie seconde pour ne pas surcharger le serveur
        sleep(0.5)
        getNextlink(root_url + next_link, list_movie)
    else:
        return list_movie

#### On peut tester la fonction pour la première page

In [7]:
r = requests.get(imdb_page)
soup = BeautifulSoup(r.text, 'lxml')
list_movie = getPageInfo(soup)

#### Pour envoyer le contenu dans une dataframe

In [16]:
df = pd.DataFrame(list_movie)

#### Pour extraire le contenu de toutes les pages

In [16]:
# Dictionnaire servant à accéder à la variable à l'intérieur d'une fonction (pas très accadémique), utile en cas d'interruption du script pour ne pas tout refaire
full_list = {}
# Liste de films, qui se constituera petit à petit
list_movie = []
getNextlink(imdb_page, list_movie)

/search/title/?title_type=feature&year=2019-01-01,2019-12-31&sort=boxoffice_gross_us,desc&start=51
/search/title/?title_type=feature&year=2019-01-01,2019-12-31&sort=boxoffice_gross_us,desc&start=101
/search/title/?title_type=feature&year=2019-01-01,2019-12-31&sort=boxoffice_gross_us,desc&start=151
/search/title/?title_type=feature&year=2019-01-01,2019-12-31&sort=boxoffice_gross_us,desc&start=201
/search/title/?title_type=feature&year=2019-01-01,2019-12-31&sort=boxoffice_gross_us,desc&start=251
/search/title/?title_type=feature&year=2019-01-01,2019-12-31&sort=boxoffice_gross_us,desc&start=301
/search/title/?title_type=feature&year=2019-01-01,2019-12-31&sort=boxoffice_gross_us,desc&start=351
/search/title/?title_type=feature&year=2019-01-01,2019-12-31&sort=boxoffice_gross_us,desc&start=401
/search/title/?title_type=feature&year=2019-01-01,2019-12-31&sort=boxoffice_gross_us,desc&start=451
/search/title/?title_type=feature&year=2019-01-01,2019-12-31&sort=boxoffice_gross_us,desc&start=501
/

KeyboardInterrupt: 

#### Dataframe complet

In [10]:
full_df = pd.DataFrame(full_list['list_movie'])

,index_,link,title,year,certificate,genre,runtime,ratings,plot,votes,gross
0,1,/title/tt4154796/,Avengers: Endgame,2019,Tous publics,"Action, Adventure, Drama",181,8.4,After the devastating events of Avengers: Infi...,"818,239",$858.37M
1,2,/title/tt6105098/,Le Roi Lion,2019,Tous publics,"Animation, Adventure, Drama",118,6.9,"After the murder of his father, a young lion p...","213,370",$543.64M
2,3,/title/tt2527338/,Star Wars: L'Ascension de Skywalker,2019,Tous publics,"Action, Adventure, Fantasy",141,6.6,The surviving members of the resistance face t...,"376,822",$515.20M
3,4,/title/tt4520988/,La Reine des Neiges 2,2019,Tous publics,"Animation, Adventure, Comedy",103,6.9,"Anna, Elsa, Kristoff, Olaf and Sven leave Aren...","138,104",$477.37M
4,5,/title/tt1979376/,Toy Story 4,2019,Tous publics,"Animation, Adventure, Comedy",100,7.8,"When a new toy called ""Forky"" joins Woody and ...","204,746",$434.04M
...,...,...,...,...,...,...,...,...,...,...,...
195,196,/title/tt9629026/,Xiao zhu pei qi guo da nian,2019,G,"Animation, Family",81,3.3,"Tang Yuan, Jiao Zi and their family enjoy Chin...",59,$0.13M
196,197,/title/tt9825006/,Avant qu'on explose,2019,,Comedy,108,6.6,The Third World War is on the horizon. Despite...,104,$0.12M
197,198,/title/tt10161238/,Geolkapseu,2019,,"Action, Comedy, Crime",107,5.9,"Mi-Young, who used to be considered the Major ...",677,$0.10M
198,199,/title/tt10550884/,The Divine Fury,2019,Tous publics,"Action, Horror, Thriller",129,6.2,An MMA fighter helps an exorcist fight evil.,"2,223",$0.10M


#### Enregistrement en csv

In [ ]:
full_df.to_csv('data.csv', index=False)

#### Si la fonction s'interrompt, on peut recommencer à partir de la dernière url

In [ ]:
# Exemple (On recopie le dernier texte qui s'affiche):
r = requests.get(root_url + '/search/title/?title_type=feature&year=2019-01-01,2019-12-31&sort=boxoffice_gross_us,desc&start=201')
list_movie = []
soup = BeautifulSoup(r.text, 'lxml')
getNextlink(imdb_page, list_movie)

In [18]:
full_df = pd.DataFrame(full_list['list_movie'])
# On dédoublonne
full_df = full_df.drop_duplicates()